In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [19]:
from keras.datasets import cifar10
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Dense, Dropout, Reshape, Input,Flatten,BatchNormalization
import numpy as np
from keras.preprocessing import image
import os,sys
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import imageio
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def load_data(data_dir, csv_file):
    df = pd.read_csv(csv_file)
    faces = os.listdir(data_dir)
    X = np.zeros((len(faces), 48,48))
    Y = np.zeros((len(faces), 1))
    for face in faces:
        fid = int(face.replace(".png",""))
        X[fid-1,:,:] = np.array(imageio.imread(data_dir+face))[:,:,0]/255.
        Y[fid-1, 0]  = int(df.loc[df["ID"]==int(fid)]["isHappy"].values)
    X  = X.reshape((-1,48,48,1))
    return (X,Y)

In [4]:
x_train, y_train = load_data("./faces/trainData/", "./faces/train_labels.csv")

In [5]:
x_train = 2.*x_train-1.

In [6]:
np.min(x_train), np.max(x_train)

(-1.0, 1.0)

In [7]:
len(x_train)

10000

In [8]:
width      = x_train.shape[1]
height     = x_train.shape[2]
channels   = x_train.shape[3]
latent_dim = 64

In [9]:
def conv(channels, f, strides=1, bn_mom=0.5, padding='same', lr=0.2):
    def layer(x):
        x = Conv2D(channels, f, strides=strides, padding = padding)(x)
        if bn_mom:
            x = BatchNormalization(momentum=bn_mom)(x)
        x = LeakyReLU(lr)(x)
        return x
    return layer

def deconv(channels, f, strides=2, bn_mom=0.5, padding='same', lr=0.01):
    def layer(x):
        x = Conv2DTranspose(channels, f, strides=strides, padding = padding)(x)
        if bn_mom:
            x = BatchNormalization(momentum=bn_mom)(x)
        x = LeakyReLU(lr)(x)
        return x
    return layer

In [10]:
generator_input = Input(shape=(latent_dim,))

x = Dense(512*3*3)(generator_input)
x = Reshape((3,3,512))(x)
x = BatchNormalization(momentum=0.5)(x)
x = LeakyReLU(0.01)(x)

x = deconv(512, 3, 2)(x)
x = deconv(256, 3, 2)(x)
x = deconv(128, 3, 2)(x)
x = deconv(64, 3, 2)(x)
x = Conv2D(channels, 5, activation='tanh', padding='same')(x)

generator = Model(generator_input, x)
generator.summary()

g_optimizer = Adam(lr=0.0001,beta_1=0.5, clipvalue=1.0, decay=1e-8)
generator.compile(optimizer=g_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4608)              299520    
_________________________________________________________________
reshape_1 (Reshape)          (None, 3, 3, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 3, 512)         2048      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 3, 3, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 6, 6, 512)         2359808   
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 6, 512)         2048      
__________

In [14]:
discriminator_input = Input(shape=(height, width, channels))

x = conv(128, 3, strides=2, bn_mom=None)(discriminator_input)
x = conv(256, 3, strides=2, bn_mom=0.5)(x)
x = conv(512, 3, strides=2, bn_mom=0.5)(x)
x = conv(1024, 3, strides=2, bn_mom=0.5)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='sigmoid')(x)

discriminator = Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = Adam(lr=0.0001, beta_1=0.5, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='categorical_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 128)       1280      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 256)       295168    
_________________________________________________________________
batch_normalization_12 (Batc (None, 12, 12, 256)       1024      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 512)         1180160   
__________

In [15]:
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))

gan = Model(gan_input, gan_output)

gan_optimizer = Adam(lr=0.00005, beta_1=0.5, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='categorical_crossentropy')

gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64)                0         
_________________________________________________________________
model_1 (Model)              (None, 48, 48, 1)         4215553   
_________________________________________________________________
model_5 (Model)              (None, 3)                 6231043   
Total params: 10,446,596
Trainable params: 4,212,609
Non-trainable params: 6,233,987
_________________________________________________________________


In [77]:
gan_optimizer = Adam(lr=0.0008, beta_1=0.5, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='categorical_crossentropy')

In [ ]:
iterations = 20000
batch_size = 100
save_dir = 'dcgan_images1_2'

a_losses = []
d_losses = []

start = 0
for step in range(iterations):
    
    #-------------------------#
    #   TRAIN DISCRIMINATOR   #
    #-------------------------#

    stop         = start+batch_size
    real_images  = x_train[start:stop]
    real_labels  = y_train[start:stop]

    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    real_targets = to_categorical(np.ones((batch_size,1))+real_labels, num_classes=3)
    real_targets += np.multiply(real_targets,0.4*np.random.randn(batch_size,3)-0.2)
    
    fake_targets = to_categorical(np.zeros((batch_size,1)), num_classes=3)
    fake_targets[:,0] += 0.4*np.random.random((batch_size,))-0.2
    
    generated_images = generator.predict(random_latent_vectors)
    
    if np.random.rand()<0.08:
        real_loss = discriminator.train_on_batch(real_images, fake_targets)
        fake_loss = discriminator.train_on_batch(generated_images, real_targets)
    else:
        real_loss = discriminator.train_on_batch(real_images, real_targets)
        fake_loss = discriminator.train_on_batch(generated_images, fake_targets)    
        
    d_loss = 0.5*(real_loss+fake_loss)

    d_losses.append(d_loss)
    
    #---------------------#
    #   TRAIN GENERATOR   #
    #---------------------#
    for i in range(np.random.randint(1,4)):
        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        misleading_targets    = to_categorical(2*np.ones((batch_size,1)))
        a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
        
    a_losses.append(a_loss)
    
    start += batch_size
    if start>(len(x_train)-batch_size):
        start = 0
    
    if step % 10 ==0:
        if not (step*batch_size)%len(x_train):
            print("\n Epoch: %d"%(step*batch_size/float(len(x_train))))
        sys.stdout.write('\r step %d; discriminator_loss: %.4f, adversarial_loss: %.4f'%(step, d_loss,a_loss))
        sys.stdout.flush()

    if not (step*batch_size)%(2*len(x_train)):
        epoch = step*batch_size/float(len(x_train))
        gan.save_weights("gan_faces.h5")
        img = image.array_to_img((generated_images[0]+1.)*127.5, scale=False)
        img.save(os.path.join(save_dir, 'generated_face'+str(step)+'.png'))
        
        #img = image.array_to_img((real_images[0]+1.)*127.5, scale=False)
        #img.save(os.path.join(save_dir, 'real_face'+str(step)+'.png'))
        
        with open("gan_loss_faces.csv","w") as f:
            w = csv.writer(f, delimiter=',')
            w.writerow(["d_loss", "a_loss"])
            for i in range(len(d_losses)):
                w.writerow([d_losses[i],a_losses[i]])

/home/qati/.anaconda3/lib/python3.5/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



 Epoch: 0
 step 90; discriminator_loss: 0.4116, adversarial_loss: 0.7058
 Epoch: 1
 step 190; discriminator_loss: 0.4423, adversarial_loss: 0.7643
 Epoch: 2
 step 290; discriminator_loss: 0.4056, adversarial_loss: 0.7157
 Epoch: 3
 step 390; discriminator_loss: 0.4354, adversarial_loss: 0.7178
 Epoch: 4
 step 490; discriminator_loss: 0.3749, adversarial_loss: 0.6949
 Epoch: 5
 step 590; discriminator_loss: 0.4952, adversarial_loss: 0.7228
 Epoch: 6
 step 690; discriminator_loss: 0.4002, adversarial_loss: 0.7029
 Epoch: 7
 step 790; discriminator_loss: 0.3948, adversarial_loss: 0.7052
 Epoch: 8
 step 890; discriminator_loss: 1.5848, adversarial_loss: 0.7437